In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

movies = [
    {
        "movie": "In the Mood for Love",
        "info": """
        Director: Wong Kar-wai
        Main Cast: Tony Leung, Maggie Cheung
        Budget / Box Office: approx. $3M / $17M
        Genre: Romantic Drama (Melodrama)
        Synopsis: In 1962 Hong Kong, two neighbors discover their spouses are having an affair and slowly develop feelings for each other, but never cross the line.
        """,
    },
    {
        "movie": "No Country for Old Men",
        "info": """
        Director: Joel Coen & Ethan Coen
        Main Cast: Tommy Lee Jones, Javier Bardem, Josh Brolin
        Budget / Box Office: approx. $25M / $171.6M
        Genre: Neo-Western Crime Thriller
        Synopsis: A man who finds a briefcase full of cash in the desert becomes the target of a relentless hitman, while a weary sheriff tries to make sense of the growing violence.
        """,
    },
    {
        "movie": "Cinema Paradiso",
        "info": """
        Director: Giuseppe Tornatore
        Main Cast: Philippe Noiret, Jacques Perrin, Salvatore Cascio
        Budget / Box Office: approx. $5M / $36M
        Genre: Coming-of-Age Drama
        Synopsis: In a small Sicilian village, a boy forms a deep friendship with the projectionist of the local movie theater, discovering his love for cinema and growing up through its memories.
        """,
    },
]

movie_prompt = PromptTemplate.from_template("Human: {movie}\nAI:{info}")

prompt = FewShotPromptTemplate(
    example_prompt=movie_prompt,
    examples=movies,
    suffix="Human: What do you know about {movie}?",
    input_variables=["movie"],
)

chain = prompt | chat

chain.invoke({"movie": "Eternal Sunshine of The Spotless Mind"})

AI:
        Director: Michel Gondry
        Main Cast: Jim Carrey, Kate Winslet
        Budget / Box Office: approx. $20M / $74.4M
        Genre: Romantic Science Fiction Drama
        Synopsis: After a painful breakup, a couple undergoes a procedure to erase each other from their memories, but as their memories are being erased, they realize their love for each other.

AIMessageChunk(content='AI:\n        Director: Michel Gondry\n        Main Cast: Jim Carrey, Kate Winslet\n        Budget / Box Office: approx. $20M / $74.4M\n        Genre: Romantic Science Fiction Drama\n        Synopsis: After a painful breakup, a couple undergoes a procedure to erase each other from their memories, but as their memories are being erased, they realize their love for each other.')